In [21]:
import logging
import random
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait as wait

import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from typing import Dict, List
from orb.utils import GetUserAgent
from orb.utils import GetProxies
from webdriver_manager.chrome import ChromeDriverManager

import logging
import random

from selenium.webdriver.common.by import By

from orb.spinner.utils.spoofing import DriverSpoofing

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

In [22]:
class ExtractMissingPersonsUrls:

    URL = "https://missingpersons.police.uk/en-gb/case-search/"

    def __init__(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.driver.get(self.URL)

        self.case_urls = []

    def ignore_warning_box(self):

        dialog_box = WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Dialog')))[0]

        close_tag = WebDriverWait(
            dialog_box, 5).until(
                ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))[-1]

        close_tag.click()

    def expand_to_100_items(self):

        dropdown_per_page = Select(WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located((By.ID, 'setPerPage')))[-1])

        dropdown_per_page.select_by_visible_text("100 per page")

    def return_page_urls(self):
        urls_banner = WebDriverWait(
            self.driver, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Pagination')))[0]

        page_urls_tags = WebDriverWait(
            urls_banner, 5).until(
                ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))[1:-2]

        self.page_urls = [self.driver.current_url]
        for tag in page_urls_tags:
            self.page_urls.append(tag.get_attribute("href"))

    def extract_case_urls(self):

        self.return_page_urls()

        for page in self.page_urls:
            if page:
                self.driver.get(page)

                case_grid = WebDriverWait(
                    self.driver, 5).until(
                        ec.visibility_of_any_elements_located((By.CLASS_NAME, 'CaseGrid')))[0]

                # Compile all cases as URLS
                case_url_tags = WebDriverWait(
                    case_grid, 5).until(
                        ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))

                for tag in case_url_tags:
                    self.case_urls.append(tag.get_attribute("href"))

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 7.05M/7.05M [00:01<00:00, 7.26MB/s]
/tmp/ipykernel_294303/2178364203.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get("https://missingpersons.police.uk/en-gb/case-search/")

In [4]:
# Accept the sensitive image warning

dialog_box = WebDriverWait(
    driver, 5).until(
        ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Dialog')))[0]

close_tag = WebDriverWait(
    dialog_box, 5).until(
        ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))[-1]

close_tag.click()

In [5]:
# Find 'per page' button and select the maximum

from selenium.webdriver.support.ui import Select

dropdown_per_page = Select(WebDriverWait(
    driver, 5).until(
        ec.visibility_of_any_elements_located((By.ID, 'setPerPage')))[-1])

dropdown_per_page.select_by_visible_text("100 per page")

In [6]:
# Find all of the pages and return the urls

urls_banner = WebDriverWait(
    driver, 5).until(
        ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Pagination')))[0]

page_urls_tags = WebDriverWait(
    urls_banner, 5).until(
        ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))[1:-2]

page_urls = [driver.current_url]
for tag in page_urls_tags:
    page_urls.append(tag.get_attribute("href"))

In [7]:
# Find the entire case grid for each url and compile a url list for all cases

case_urls = []
for page in page_urls:

    if page:
        driver.get(page)

        case_grid = WebDriverWait(
            driver, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'CaseGrid')))[0]

        # Compile all cases as URLS
        case_url_tags = WebDriverWait(
            case_grid, 5).until(
                ec.visibility_of_any_elements_located((By.TAG_NAME, 'a')))

        for tag in case_url_tags:
            case_urls.append(tag.get_attribute("href"))

In [16]:


def build_raw_case_dict() -> Dict[str, str]:

    case_date_dict = {}

    # Find the case number
    case_number = WebDriverWait(
        driver, 5).until(
            ec.visibility_of_any_elements_located((By.CLASS_NAME, 'PageTitle')))[0].text
    case_date_dict["CASE_NUMBER"] = case_number


    # Find and extract case data
    case_data_raw = WebDriverWait(
        driver, 5).until(
            ec.visibility_of_any_elements_located((By.CLASS_NAME, 'CaseData')))[0]

    # All row entries are broken down into divs
    # These divs are split into further divs consisting of 'keys' and 'values'
    case_data_rows = WebDriverWait(
        case_data_raw, 5).until(
            ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Entry')))


    for entry in case_data_rows:
        key_ = WebDriverWait(
            entry, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Key')))[0].text

        value_ = WebDriverWait(
            entry, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Value')))[0].text

        case_date_dict[key_.upper().replace(" ", "_")] = value_

    # Extract location information
    location_banner = WebDriverWait(
        driver, 5).until(
            ec.visibility_of_any_elements_located((By.CLASS_NAME, 'CaseMap')))[0]
    
    try:
        road = WebDriverWait(
            location_banner, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Road')))[0].text.lower()
        case_date_dict['LOCATION_ROAD'] = road
    except Exception as e:
        case_date_dict['LOCATION_ROAD'] = None
        
    try:
        county = WebDriverWait(
            location_banner, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'County')))[0].text.lower()
        case_date_dict['LOCATION_COUNTY'] = county
    except Exception as e:
        case_date_dict['LOCATION_COUNTY'] = None

    try:
        country = WebDriverWait(
            location_banner, 5).until(
                ec.visibility_of_any_elements_located((By.CLASS_NAME, 'Country')))[0].text.lower()
        case_date_dict['LOCATION_COUNTRY'] = country
    except Exception as e:
        case_date_dict['LOCATION_COUNTRY'] = None

    # Attempt to find who found them
    try:
        finders = WebDriverWait(
            location_banner, 5).until(
                ec.visibility_of_any_elements_located((By.TAG_NAME, 'strong')))[0].text.lower()
        case_date_dict['FINDING_PARTY'] = finders
    except Exception as e:
        case_date_dict['FINDING_PARTY'] = None

    return case_date_dict

In [17]:
import string
from datetime import datetime

def clean_case_number(case_number_str: str) -> str:
    """
    Cleans the case number to return a serialised code.
    Args:
        case_number_str: string of the case number scraped from site
    Returns:
        Formatted case number string
    """
    return ''.join(char for char in case_number_str if char in string.digits + string.punctuation)

def split_age_range(age_range_str: str) -> Dict[str, int]:
    """
    Splits the age range into a minimum and maximum integar value.
    Args:
        age_range_str: String of the age range
    Returns:
        Dictionary of the min and maximum age values in the folm of {'MIN_AGE': x, 'MAX_AGE': y}
    """
    split_values = [age for age in '18 - 30'.split() if age.isdigit()]
    return {"MIN_AGE": min(split_values), "MAX_AGE": max(split_values)}

def extract_height_cm(height_str: str) -> int:
    """
    Returns the height in cm.
    Args:
        height_str: String value of the height (typially containing cm and ft/inches)
    Returns:
        Integar of the height in cm
    """
    return height_str[:height_str.find('cm')]

def format_date_found(date_found: str) -> datetime.date:
    """
    Formats a string date into a date object
    Args:
        date_found: string for the date found, typically in the "%d %B %Y" format
    Returns:
        Date object
    """
    date_format = "%d %B %Y"
    return datetime.strptime(date_found, date_format).date()

def rough_format_distinguishing_features(dist_features_str: str) -> Dict[str, str]:
    """
    One of the more complex engineering tasks and requires a closer analysis.
    We separate this field into a list where possible.

    Typically, for distingusihing features these come as the following (split by a '-'):

        - CATEGORY: Represents the general category of the entry.
        - SUBCATEGORY: Provides additional details about the entry if available.
        - LOCATION: Indicates the specific body part or location mentioned in the entry.
        - DESCRIPTION: Describes any additional details or information related to the entry.

    Although this is not always the case, often the 'Description' field is missing.
    In some cases this structure is not adhered to whatsoever and it is just a free text input.
    
    """
    return_dict = {}

    # Split the entry string into a list by the new line character
    descriptors_list = [i for i in dist_features_str.split('\n') if len(i) > 1]

    for n, entry in enumerate(descriptors_list, start=1):
        # Attempt to split into the categories
        category_items = [i.strip().upper() for i in entry.split('-') if len(i) > 1]

        if len(category_items) < 2:
            continue

        category_dict = {
            "CATEGORY": None,
            "SUBCATEGORY": None,
            "LOCATION": None,
            "DESCRIPTION": None,
        }

        for j, key in zip(category_items, category_dict.keys()):
            category_dict[key] = j

        return_dict[f"FEATURE_{n}"] = category_dict

    return return_dict


def rough_format_clothing(clothing_str: str) -> Dict[str, str]:
    """
    One of the more complex engineering tasks and requires a closer analysis.
    We separate this field into a list where possible.

    Typically, for clothing these come as the following (split by a '-'):

        - CATEGORY: Represents the general category type of the entry (footwear etc).
        - SUBCATEGORY: Provides additional details about the entry if available (shoes if CATEGORY is footwear for example).
        - COLOUR: Colour of the CATEGORY/SUBCATEGORY.
        - PATTERN: Describes the pattern of the entry.
        - DESCRIPTION: Describes any additional details or information related to the entry. 

    Although this is not always the case, often the 'Description' field is missing.
    In some cases this structure is not adhered to whatsoever and it is just a free text input.
    """
    return_dict = {}

    # Split the entry string into a list by the new line character
    descriptors_list = [i for i in clothing_str.split('\n') if len(i) > 1]

    for n, entry in enumerate(descriptors_list, start=1):
        # Attempt to split into the categories
        category_items = [i.strip().upper() for i in entry.split('-') if len(i) > 1]

        if len(category_items) < 2:
            continue

        category_dict = {
            "CATEGORY": None,
            "SUBCATEGORY": None,
            "COLOUR": None,
            "PATTERN": None,
            "DESCRIPTION": None,
        }

        for j, key in zip(category_items, category_dict.keys()):
            category_dict[key] = j

        return_dict[f"CLOTHING_{n}"] = category_dict

    return return_dict


def format_possessions(possessions_str: str) -> List[str]:

    split_char = ','
    if '(1)' in possessions_str:
        split_char = '(1)'

    return [i.strip().lower() for i in possessions_str.split(split_char) if len(i) > 1]

In [18]:
def format_raw_case_dict(case_date_dict: Dict[str, str]) -> Dict[str, str]:

    case_dict_clean = {}

    # Clean case number
    case_dict_clean['CASE_NUMBER'] = clean_case_number(case_number_str=case_date_dict['CASE_NUMBER'])

    # Clean gender value
    case_dict_clean['GENDER'] = case_date_dict['GENDER'].lower()

    # Clean age ranges
    case_dict_clean.update(split_age_range(age_range_str=case_date_dict['AGE_RANGE']))

    # Clean ethnicity
    if case_date_dict('ETHNICITY'):
        case_dict_clean['ETHNICITY'] = case_date_dict['ETHNICITY'].lower()
    else:
        case_dict_clean['ETHNICITY'] = None

    # Clean Height
    if case_date_dict.get('HEIGHT'):
        case_dict_clean['HEIGHT_CM'] = extract_height_cm(height_str=case_date_dict['HEIGHT'])
    else:
        case_dict_clean['HEIGHT_CM'] = None

    # Clean build
    if case_date_dict.get('BUILD'):
        case_dict_clean['BUILD'] = case_date_dict['BUILD'].lower()
    else:
        case_dict_clean['BUILD'] = None

    # Format date found
    case_dict_clean['DATE_FOUND'] = format_date_found(date_found=case_date_dict['DATE_FOUND'])

    # Format estimated death date
    if case_date_dict.get('ESTIMATED_DEATH'):
        case_dict_clean['ESTIMATED_DEATH'] = format_date_found(date_found=case_date_dict['ESTIMATED_DEATH'])
    else:
        case_dict_clean['ESTIMATED_DEATH'] = None

    # Format body found state
    case_dict_clean['BODY_OR_REMAINS'] = case_date_dict['BODY_OR_REMAINS'].lower()

    # Format hair
    if case_date_dict.get('HAIR'):
        case_dict_clean['HAIR'] = case_date_dict['HAIR'].lower()
    else:
        case_dict_clean['HAIR'] = None

    # Format facial hair
    if case_date_dict.get('FACIAL_HAIR'):
        case_dict_clean['FACIAL_HAIR'] = case_date_dict['FACIAL_HAIR'].lower()
    else:
        case_dict_clean['FACIAL_HAIR'] = None

    # Format eye colour
    case_dict_clean['EYE_COLOUR'] = case_date_dict['EYE_COLOUR'].lower()

    if case_date_dict.get('DISTINGUISHING_FEATURES'):
        case_dict_clean['DISTINGUISHING_FEATURES'] = rough_format_distinguishing_features(
                case_date_dict['DISTINGUISHING_FEATURES']
        )
    else:
        case_dict_clean['DISTINGUISHING_FEATURES'] = None

    if case_date_dict.get('CLOTHING'):
        case_dict_clean['CLOTHING'] = rough_format_clothing(case_date_dict['CLOTHING'])
    else:
        case_dict_clean['CLOTHING'] = None

    if case_date_dict.get('POSSESSIONS'):
        case_dict_clean['POSSESSIONS'] = format_possessions(possessions_str=case_date_dict['POSSESSIONS'])
    else:
        case_dict_clean['POSSESSIONS'] = None

    # Clean circumstances value
    if case_date_dict.get('CIRCUMSTANCES'):
        case_dict_clean['CIRCUMSTANCES'] = case_date_dict['CIRCUMSTANCES'].lower()
    else:
        case_dict_clean['CIRCUMSTANCES'] = None

    # Clean JEWELLERY value
    if case_date_dict.get('JEWELLERY'):
        case_dict_clean['JEWELLERY'] = case_date_dict['JEWELLERY'].lower()
    else:
        case_dict_clean['JEWELLERY'] = None

    # Clean known_not_to_be value
    if case_date_dict.get('KNOWN_NOT_TO_BE'):
        case_dict_clean['KNOWN_NOT_TO_BE'] = case_date_dict['KNOWN_NOT_TO_BE'].lower()
    else:
        case_dict_clean['KNOWN_NOT_TO_BE'] = None


    return case_dict_clean


In [11]:
full_df = pd.DataFrame()

for n, url in enumerate(case_urls, start=1):

    print(f"Running for URL {n} out of {len(case_urls)}")

    driver.get(url)

    # Build raw dict
    raw_case_dict = build_raw_case_dict()

    # Build cleaned dict
    clean_case_dict = format_raw_case_dict(case_date_dict=raw_case_dict)

    # Build single data frame row
    row_df = pd.DataFrame([clean_case_dict])

    # Concatenate row_df to full_df
    full_df = pd.concat([full_df, row_df], ignore_index=True)

Running for URL 1 out of 555
Running for URL 2 out of 555
Running for URL 3 out of 555
Running for URL 4 out of 555
Running for URL 5 out of 555
Running for URL 6 out of 555
Running for URL 7 out of 555
Running for URL 8 out of 555
Running for URL 9 out of 555
Running for URL 10 out of 555
Running for URL 11 out of 555
Running for URL 12 out of 555
Running for URL 13 out of 555
Running for URL 14 out of 555
Running for URL 15 out of 555
Running for URL 16 out of 555
Running for URL 17 out of 555
Running for URL 18 out of 555
Running for URL 19 out of 555
Running for URL 20 out of 555
Running for URL 21 out of 555
Running for URL 22 out of 555
Running for URL 23 out of 555
Running for URL 24 out of 555
Running for URL 25 out of 555
Running for URL 26 out of 555
Running for URL 27 out of 555
Running for URL 28 out of 555
Running for URL 29 out of 555
Running for URL 30 out of 555
Running for URL 31 out of 555
Running for URL 32 out of 555
Running for URL 33 out of 555
Running for URL 34 

KeyError: 'ETHNICITY'

In [19]:
len(full_df)

523

In [20]:
full_df

,CASE_NUMBER,GENDER,MIN_AGE,MAX_AGE,ETHNICITY,HEIGHT_CM,BUILD,DATE_FOUND,ESTIMATED_DEATH,BODY_OR_REMAINS,HAIR,FACIAL_HAIR,EYE_COLOUR,DISTINGUISHING_FEATURES,CLOTHING,POSSESSIONS,CIRCUMSTANCES,JEWELLERY,KNOWN_NOT_TO_BE
0,23-001473,male,18,30,afro caribbean,180,slim,2022-12-14,None,body,black,None,brown,"{'FEATURE_1': {'CATEGORY': 'LACKING', 'SUBCATE...","{'CLOTHING_1': {'CATEGORY': 'TROUSERS', 'SUBCA...",None,None,None,None
1,23-001476,male,18,30,afro caribbean,172,None,2022-12-14,None,body,black - short,None,brown,"{'FEATURE_1': {'CATEGORY': 'PECULIARITY', 'SUB...","{'CLOTHING_1': {'CATEGORY': 'FOOTWEAR', 'SUBCA...","[tobacco, cigarette lighters, tobacco papers, ...",None,None,None
2,23-001464,male,18,30,unknown,None,None,2022-12-14,None,body,black,None,brown,None,"{'CLOTHING_1': {'CATEGORY': 'TROUSERS', 'SUBCA...",None,None,None,None
3,23-000490,male,18,30,afro caribbean,177,slim,2022-12-07,None,body,black - short,None,unknown,None,"{'CLOTHING_1': {'CATEGORY': 'FOOTWEAR', 'SUBCA...",[a leather band on his right wrist and left an...,None,bracelet - 2 - leather band worn on right wris...,None
4,21-006966,male,18,30,white european,175,medium,2021-09-23,2021-09-23,body,None,None,unknown,"{'FEATURE_1': {'CATEGORY': 'PECULIARITY', 'SUB...","{'CLOTHING_1': {'CATEGORY': 'FOOTWEAR', 'SUBCA...",None,deceased male found floating in the english ch...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,06-026171,male,18,30,unknown,175,thin,1971-04-18,None,body,None,None,blue,None,"{'CLOTHING_1': {'CATEGORY': 'FOOTWEAR', 'SUBCA...",None,burnt body found at barnes passage w4.,None,None
519,06-024612,male,18,30,white european,175,medium,1971-03-31,None,body,black - curly,None,unknown,"{'FEATURE_1': {'CATEGORY': 'PECULIARITY', 'SUB...","{'CLOTHING_1': {'CATEGORY': 'HOSE', 'SUBCATEGO...",None,"found on the foreshore of the severn estuary, ...",None,None
520,06-024611,male,18,30,white european,172,thin,1971-03-26,None,body,brown - straight - short,None,unknown,"{'FEATURE_1': {'CATEGORY': 'PECULIARITY', 'SUB...","{'CLOTHING_1': {'CATEGORY': 'HOSE', 'SUBCATEGO...",None,male body was found in a shallow grave in a fi...,ring - 1 - ladies 9ct gold wedding ring on rig...,None
521,06-026172,male,18,30,unknown,178,thin,1971-01-28,None,body,grey - bald,None,unknown,None,"{'CLOTHING_1': {'CATEGORY': 'JUMPER', 'SUBCATE...",None,decomposed body found in a derelict shed at th...,None,None
